In [2]:
# Count type 2

In [3]:
import sys
import mpi_slingshot as sl
from mpi_slingshot import write_and_run

In [22]:
VERSION='v4'
DEFAULT_CORPUS='HathiEssays'
CORPUS=sys.argv[1] if len(sys.argv)>1 and not sys.argv[1].startswith('-') else DEFAULT_CORPUS
print(f'Scoring corpus: {CORPUS}')

Scoring corpus: HathiEssays


In [23]:
# print(sys.argv)
# stop

['/home/ryan/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py', '-f', '/home/ryan/.local/share/jupyter/runtime/kernel-fc9b6bce-f31c-4036-8697-8bd43f665a1b.json']


NameError: name 'stop' is not defined

In [8]:
import lltk
C=lltk.load(CORPUS)

In [21]:
%%write_and_run .count.slingshot.v2.py

### SLINGSHOT

import sys
sys.path.append('../..')
from abslithist.words import *
import ujson as json

dfnorms=get_allnorms()

def score_freqs(inpd):
    path_freqs=inpd['path_freqs']
    try:
        with open(path_freqs) as f: freqs=json.load(f)
    except Exception as e:
        print('!',e)
        return []
    freql=[w for w,c in freqs.items() for i in range(c)]
    freqdf=pd.DataFrame(index=freql)
    
    # by col
#     odf=pd.DataFrame()
#     for col in dfnorms.columns:
#         scol=dfnorms[col]
#         scoldf=freqdf.join(scol.dropna().to_frame(),how='inner')
#         # make avg
#         avgdf=pd.DataFrame(dict((x,scoldf.agg(x)) for x in ['mean','median','sum','std','size']))
#         odf=odf.append(avgdf)
#     return odf.rename_axis('norm').reset_index().to_dict('records')
    bothdf=freqdf.join(dfnorms)
    return dict(bothdf.mean())

In [10]:
# prepare objects
objects = [
    {'id':t.id, 'path_freqs':t.path_freqs}
    for t in C.texts()
]

In [17]:
# objects[0]
# paths=[x['path_freqs'] for x in objects]
# paths[0]

In [18]:
# Counter([x.split('/corpora/')[-1].split('/freqs/')[0] for x in paths])

In [19]:
# !ls /home/ryan/lltk_data/corpora/ecco_tcp/freqs/K051430.000.json

In [20]:
# score_freqs(objects[0])

## Map

In [9]:
# Slingshot!

savedir=f'../../data/scores/v5/{C.name}'

cmd = sl.shoot(
    func='score_freqs',
    path_src='.count.slingshot.v2.py',
    objects=objects,
    savedir=savedir,
    parallel=7
#     overwrite=True
)
cmd

[Slingshot] Preparing input: 100%|██████████| 1809/1809 [00:00<00:00, 168862.44it/s]


'slingshot -code ../../data/scores/v4/CanonFiction/func.py -func score_freqs -pathlist ../../data/scores/v4/CanonFiction/input.jsonl -savedir ../../data/scores/v4/CanonFiction -resume -parallel 4 -resume -nosave'

In [10]:
!{cmd}

## Reduce

In [15]:
old=[]
for metadx,dx in sl.stream_results(os.path.join(savedir,'cache')):
    if type(dx)!=dict: continue
    idx=metadx['id']
    for k,v in dx.items():
        odx={'id':idx}
        odx['contrast'],odx['source'],odx['period']=k.split('.')
        old.append(odx)
#     break
df=pd.DataFrame(old)
df

[Slingshot] Reading results.rank=0003.jsonl: 100%|██████████| 452/452 [00:00<00:00, 14629.64it/s]


,id,contrast,source,period
0,markmark.Jackson.The_Haunting_of_Hill_Hous.1515,Abs-Conc,LSN-Hapt,C16
1,markmark.Jackson.The_Haunting_of_Hill_Hous.1515,Abs-Conc,LSN-Imag,C16
2,markmark.Jackson.The_Haunting_of_Hill_Hous.1515,Abs-Conc,MRC-Conc,C16
3,markmark.Jackson.The_Haunting_of_Hill_Hous.1515,Abs-Conc,MRC-Imag,C16
4,markmark.Jackson.The_Haunting_of_Hill_Hous.1515,Abs-Conc,MT-Conc,C16
...,...,...,...,...
101243,chadwyck.Flint.The_Life_and_Adventures_o.0260,Abs-Conc,MRC-Imag,orig
101244,chadwyck.Flint.The_Life_and_Adventures_o.0260,Abs-Conc,MT-Conc,orig
101245,chadwyck.Flint.The_Life_and_Adventures_o.0260,Abs-Conc,PAV-Conc,orig
101246,chadwyck.Flint.The_Life_and_Adventures_o.0260,Abs-Conc,PAV-Imag,orig


In [16]:
df.to_feather(f'../../data/scores/data.corpus_scores.{VERSION}.{C.name}.ft')